In [ ]:
SELECT current_database() AS DATABASE_NAME, current_schema() AS SCHEMA_NAME

In [ ]:
# Import python packages
from snowflake.core import Root

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

session.use_role("FRED_ROLE")
session.use_warehouse("FRED_WH")

database_name = "FRED_DB"
schema_name = "FRED_SCHEMA"

session.use_schema(f"{database_name}.{schema_name}")

In [ ]:
# Load datasets from Snowflake table
dgs10 = session.table("FRED_10Y_TABLE").to_pandas()
dgs2 = session.table("FRED_2Y_TABLE").to_pandas()  

# Convert DATE column to datetime
dgs10['DATE'] = pd.to_datetime(dgs10['OBS_DATE'])
dgs2['DATE'] = pd.to_datetime(dgs2['OBS_DATE'])

dgs_df = pd.merge(dgs10, dgs2, on='OBS_DATE', suffixes=('_10y', '_2y'))

# Convert yields to numeric (handling missing or invalid values)
dgs_df['DGS10'] = pd.to_numeric(dgs_df['PERCENT_CHANGE_10y'], errors='coerce')
dgs_df['DGS2'] = pd.to_numeric(dgs_df['PERCENT_CHANGE_2y'], errors='coerce')

# Calculate the spread
dgs_df['Spread'] = dgs_df['DGS10'] - dgs_df['DGS2']

# Add columns for Month and Year
dgs_df['OBS_DATE'] = pd.to_datetime(dgs_df['OBS_DATE'], errors='coerce')
dgs_df['Month'] = dgs_df['OBS_DATE'].dt.to_period('M')  
dgs_df['Year'] = dgs_df['OBS_DATE'].dt.year

# Group by Month and calculate average spread
monthly_spread = dgs_df.groupby('Month')['Spread'].mean().reset_index()
# Group by Year and calculate average spread
yearly_spread = dgs_df.groupby('Year')['Spread'].mean().reset_index()

# Identify months/years with inverse yield (negative spread)
monthly_inverse_yield = monthly_spread[monthly_spread['Spread'] < 0]
yearly_inverse_yield = yearly_spread[yearly_spread['Spread'] < 0]

print("Monthly Inverse Yield Curve:")
print(monthly_inverse_yield)

print("Yearly Inverse Yield Curve:")
print(yearly_inverse_yield)